In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook, tqdm
import sys
from copy import deepcopy

import utils

git_dir = utils.get_git_root(os.getcwd())
simulate_dir = os.path.join(git_dir, 'simulate')
data_dir = os.path.join(git_dir, 'data')
output_dir = os.path.join(git_dir, 'output')

metadata_csv_path = os.path.join(data_dir, utils.METADATA_FILENAME + "." + utils.INPUT_DATA_FORMAT)
training_csv_path = os.path.join(data_dir, utils.TRAINING_FILENAME + "." + utils.INPUT_DATA_FORMAT)

if git_dir not in sys.path:
    sys.path.append(git_dir)
    
if simulate_dir not in sys.path:
    sys.path.append(simulate_dir)

In [2]:
metadata = pd.read_csv(metadata_csv_path, sep=";", index_col = 0).sort_index()
training_df = pd.read_csv(training_csv_path, sep = ";")

## Simulation Class

In [3]:
class Simulation(object):
    """ Handles running a simulation.
    """
    def __init__(self,
                 data,
                 battery,
                 site_id):
        """ Creates initial simulation state based on data passed in.
            :param data: contains all the time series needed over the considered period
            :param battery: is a battery instantiated with 0 charge and the relevant properties
            :param site_id: the id for the site (building)
        """

        self.data = data

        # building initialization
        self.actual_previous_load = self.data.actual_consumption.values[0]
        self.actual_previous_pv = self.data.actual_pv.values[0]

        # align actual as the following, not the previous 15 minutes to
        # simplify simulation
        self.data.loc[:, 'actual_consumption'] = self.data.actual_consumption.shift(-1)
        self.data.loc[:, 'actual_pv'] = self.data.actual_pv.shift(-1)

        self.site_id = site_id
        self.load_columns = data.columns.str.startswith('load_')
        self.pv_columns = data.columns.str.startswith('pv_')
        self.price_sell_columns = data.columns.str.startswith('price_sell_')
        self.price_buy_columns = data.columns.str.startswith('price_buy_')

        # initialize money at 0.0
        self.money_spent = 0.0
        self.money_spent_without_battery = 0.0

        # battery initialization
        self.battery = battery

    def run(self):
        """ Executes the simulation by iterating through each of the data points
            It returns both the electricity cost spent using the battery and the
            cost that would have been incurred with no battery.
        """
        battery_controller = BatteryContoller()

        for current_time, timestep in self.data.iterrows(): #tqdm(self.data.iterrows(), total=self.data.shape[0], desc=' > > > > timesteps\t'):
            # can't calculate results without actual, so skip (should only be last row)
            print("going into simulate_timestep")
            print(current_time)
            if pd.notnull(timestep.actual_consumption):
                
                self.simulate_timestep(battery_controller, current_time, timestep)

        return self.money_spent, self.money_spent_without_battery

    def simulate_timestep(self, battery_controller, current_time, timestep):
        """ Executes a single timestep using `battery_controller` to get
            a proposed state of charge and then calculating the cost of
            making those changes.
            :param battery_controller: The battery controller
            :param current_time: the timestamp of the current time step
            :param timestep: the data available at this timestep
        """
        # get proposed state of charge from the battery controller
        proposed_state_of_charge = battery_controller.propose_state_of_charge(
            self.site_id,
            current_time,
            deepcopy(self.battery),
            self.actual_previous_load,
            self.actual_previous_pv,
            timestep[self.price_buy_columns],
            timestep[self.price_sell_columns],
            timestep[self.load_columns],
            timestep[self.pv_columns]
        )

        # get energy required to achieve the proposed state of charge
        grid_energy, battery_energy_change = self.simulate_battery_charge(self.battery.current_charge,
                                                                          proposed_state_of_charge,
                                                                          timestep.actual_consumption,
                                                                          timestep.actual_pv)
        # timestep 1: grid_energy 51414.276550899995 battery_energy_change 17812.5
        # timestep 2: grid_energy -27026.2524042 battery_energy_change 17812.5
        # timestep 3: grid_energy -25193.864794499998 battery_energy_change 17812.5
        
        # timestep 1: actual_consumption 89777.05193839999 actual_pv 57112.7753875
        # timestep 2: actual_consumption 90821.8188648 actual_pv 136598.071269
        # timestep 3: actual_consumption 93881.15636749999 actual_pv 137825.021162
        grid_energy_without_battery = timestep.actual_consumption - timestep.actual_pv
        # timestep 1: grid_energy_without_battery = 89777.05193839999 - 57112.7753875 = 32664.27655089999
        # timestep 2: grid_energy_without_battery = 90821.8188648 - 136598.071269 = -45776.2524042
        # timestep 3: grid_energy_without_battery = 93881.15636749999 - 137825.021162 = -43943.86479450001
        
        # buy or sell energy depending on needs
        # timestep 1: price_buy_00 = 0.031 price_sell_00 = 0.031
        # timestep 2: price_buy[0] 0.049 price_sell[0] 0.031
        # timestep 3: price_buy[0] 0.049 price_sell[0] 0.031
        price = timestep.price_buy_00 if grid_energy >= 0 else timestep.price_sell_00
        # timestep 1: price = price_buy_00 = 0.031
        # timestep 2: price = price_sell_00 = 0.031
        # timestep 3: price = price_sell_00 = 0.031
        
        price_without_battery = timestep.price_buy_00 if grid_energy_without_battery >= 0 else timestep.price_sell_00
        # timestep 1: price_without_battery = price_buy_00 = 0.031
        # timestep 2: price_without_battery = price_sell_00 = 0.031
        # timestep 3: price_without_battery = price_sell_00 = 0.031
        
        # calculate spending based on price per kWh and energy per Wh
        self.money_spent += grid_energy * (price / 1000.)
#         print("self.money_spent", self.money_spent)
        # timestep 1 = 51414.276550899995 * ( 0.031 / 1000 ) = 1.5938425730779
        # timestep 2 = -27026.2524042 * ( 0.031 / 1000 ) = -0.8378138245302 + 1.5938425730779 = 0.7560287485477
        # timestep 3 = -25193.864794499998 * ( 0.031 / 1000 ) = -0.7810098086295 + 0.7560287485477 = -0.0249810600818
        
        
        self.money_spent_without_battery += grid_energy_without_battery * (price_without_battery / 1000.)
#         print("self.money_spent_without_battery", self.money_spent_without_battery)
        # timestep 1 = 32664.27655089999 * ( 0.031 / 1000) = 1.0125925730779
        # timestep 2 = -45776.2524042 * (0.031 / 1000) = -1.4190638245302 + 1.0125925730779 = -0.4064712514523
        # timestep 3 = -43943.86479450001 * (0.031 / 1000) = -1.3622598086295 + -0.4064712514523 = -1.7687310600818
        
        # update current state of charge
        self.battery.current_charge += battery_energy_change / self.battery.capacity
        # timestep 1 = 17812.5 / 300000 = 0.059375
        # timestep 2 = 17812.5 / 300000 = 0.059375 + 0.059375 = 0.11875
        # timestep 3 = 17812.5 / 300000 = 0.059375 + 0.11875 = 0.178125
        
        self.actual_previous_load = timestep.actual_consumption
        self.actual_previous_pv = timestep.actual_pv

    def simulate_battery_charge(self, initial_state_of_charge, proposed_state_of_charge, actual_consumption, actual_pv):
        """ Charges or discharges the battery based on what is desired and
            available energy from grid and pv.
            :param initial_state_of_charge: the current state of the battery
            :param proposed_state_of_charge: the proposed state for the battery
            :param actual_consumption: the actual energy consumed by the building
            :param actual_pv: the actual pv energy produced and available to the building
        """
#         print("initial_state_of_charge", initial_state_of_charge)
#         print("proposed_state_of_charge", proposed_state_of_charge)
#         print("actual_consumption", actual_consumption)
#         print("actual_pv", actual_pv)
#         print("self.battery.capacity", self.battery.capacity)
#         print("self.battery.charging_efficiency", self.battery.charging_efficiency)
#         print("self.battery.discharging_efficiency", self.battery.discharging_efficiency)
#         print("self.battery.discharging_power_limit", self.battery.discharging_power_limit)
#         print("self.battery.charging_power_limit", self.battery.charging_power_limit)

        # charge is bounded by what is feasible
        proposed_state_of_charge = np.clip(proposed_state_of_charge, 0.0, 1.0) #1
        
        # calculate proposed energy change in the battery
        target_energy_change = (proposed_state_of_charge - initial_state_of_charge) * self.battery.capacity 
        # timestep 1: (1 - 0) * 300000 = 300000
        # timestep 2: (1 - 0.059375) * 300000 = 282187.5
        # timestep 3: (1 - 0.11875) * 300000 = 264375

        # efficiency can be different whether we intend to charge or discharge
        if target_energy_change >= 0:
            efficiency = self.battery.charging_efficiency # .95
            # assuming we multiply by .25 bc efficiency is based on per-hour basis, while simulation ran every 15 mins?
            target_charging_power = target_energy_change / ((15. / 60.) * efficiency)  
            # timestep 1 = 300000 / ((15. / 60.) * 0.95) = 1263157.894736842105263
            # timestep 2 = 282187.5 / ((15. / 60.) * 0.95) = 1188157.894736842105263
            # timestep 3 = 264375 / ((15. / 60.) * 0.95) = 1113157.894736842105263
        else:
            efficiency = self.battery.discharging_efficiency
            target_charging_power = target_energy_change * efficiency / (15. / 60.)

        # actual power is bounded by the properties of the battery
        
        # GB: The max amount target_charging_power is bounded by the (dis)charging_power_limit. 
        # target_charging_power is determined by the target_energy_change. 
        # So there's a practical max of the differences in states of percentage change of the battery charge.
        # In the case of Site ID 1, Battery 1, 
        actual_charging_power = np.clip(target_charging_power,
                                        self.battery.discharging_power_limit,
                                        self.battery.charging_power_limit)
        # timestep 1: 75000
        # timestep 2: 75000
        # timestep 3: 75000

        # actual energy change is based on the actual power possible and the efficiency
        if actual_charging_power >= 0:
            actual_energy_change = actual_charging_power * (15. / 60.) * efficiency
            # timestep 1: 75000 * ((15. / 60.) * .95) = 17812.5
            # timestep 2: 75000 * ((15. / 60.) * .95) = 17812.5
            # timestep 3: 75000 * ((15. / 60.) * .95) = 17812.5
        else:
            actual_energy_change = actual_charging_power * (15. / 60.) / efficiency

        # what we need from the grid = (the power put into the battery + the consumption) - what is available from pv
        grid_energy = (actual_charging_power * (15. / 60.) + actual_consumption) - actual_pv 
        # timestep 1: (75000 * (15. / 60.) + 89777.05193839999) - 57112.7753875 =  51414.27655089999
        # timestep 2: (75000 * (15. / 60.)) + 90821.8188648 - 136598.071269 = -27026.2524042
        # timestep 3: (75000 * (15. / 60.)) + 93881.15636749999 - 137825.021162 = -25193.86479450001
        # if positive, we are buying from the grid; if negative, we are selling 
#         print("grid_energy", grid_energy)
        # timestep 1: 51414.27655089999
        # timestep 2: -27026.2524042
        # timestep 3: -25193.86479450001
        
#         print("actual_energy_change", actual_energy_change) 
        # timestep 1: 17812.5
        # timestep 2: 17812.5
        # timestep 3: 17812.5
        
        return grid_energy, actual_energy_change


## Battery Class

In [4]:
class Battery(object):
    """ Used to store information about the battery.
       :param current_charge: is the initial state of charge of the battery
       :param capacity: is the battery capacity in Wh
       :param charging_power_limit: the limit of the power that can charge the battery in W
       :param discharging_power_limit: the limit of the power that can discharge the battery in W
       :param battery_charging_efficiency: The efficiecny of the battery when charging
       :param battery_discharing_efficiecny: The discharging efficiency
    """
    def __init__(self,
                 current_charge=0.0,
                 capacity=0.0,
                 charging_power_limit=1.0,
                 discharging_power_limit=-1.0,
                 charging_efficiency=0.95,
                 discharging_efficiency=0.95):
        self.current_charge = current_charge
        self.capacity = capacity
        self.charging_power_limit = charging_power_limit
        self.discharging_power_limit = discharging_power_limit
        self.charging_efficiency = charging_efficiency
        self.discharging_efficiency = discharging_efficiency

## BatteryController Class - First Place

In [5]:
class FirstPlaceBatteryContoller(object):
    """ The BatteryContoller class handles providing a new "target state of charge"
        at each time step.
        This class is instantiated by the simulation script, and it can
        be used to store any state that is needed for the call to
        propose_state_of_charge that happens in the simulation.
        The propose_state_of_charge method returns the state of
        charge between 0.0 and 1.0 to be attained at the end of the coming
        quarter, i.e., at time t+15 minutes.
        The arguments to propose_state_of_charge are as follows:
        :param site_id: The current site (building) id in case the model does different work per site
        :param timestamp: The current timestamp inlcuding time of day and date
        :param battery: The battery (see battery.py for useful properties, including current_charge and capacity)
        :param actual_previous_load: The actual load of the previous quarter.
        :param actual_previous_pv_production: The actual PV production of the previous quarter.
        :param price_buy: The price at which electricity can be bought from the grid for the
          next 96 quarters (i.e., an array of 96 values).
        :param price_sell: The price at which electricity can be sold to the grid for the
          next 96 quarters (i.e., an array of 96 values).
        :param load_forecast: The forecast of the load (consumption) established at time t for the next 96
          quarters (i.e., an array of 96 values).
        :param pv_forecast: The forecast of the PV production established at time t for the next
          96 quarters (i.e., an array of 96 values).
        :returns: proposed state of charge, a float between 0 (empty) and 1 (full).
    """
    
    step = 960
    
    def propose_state_of_charge(self,
                                site_id,
                                timestamp,
                                battery,
                                actual_previous_load,
                                actual_previous_pv_production,
                                price_buy,
                                price_sell,
                                load_forecast,
                                pv_forecast):

        # return the proposed state of charge ...
        
# Step 1: Create the variables.
######## current_charge
######## expected load in next 15 mins
######## expected pv to add to charge in next 15 mins
######## price to buy energy if PV + Storage < Load
######## price to sell energy if PV + Storage > Load, and 
# Step 2: Define the constraints.
# Step 3: Define the objective function.
# Step 4: Declare the solver—the method that implements an algorithm for finding the optimal solution.
# Step 5: Invoke the solver and display the results.     

        # divide by 4 bc power_limit is based on hour, timestep @ 15 mins
#         timestep_max_charge = (battery.charging_power_limit * battery.charging_efficiency) / 4.
#         timestep_max_discharge = (battery.discharging_power_limit * battery.discharging_efficiency) / 4.
        
#         grid_energy = actual_previous_load - actual_previous_pv_production
        
#         current_grid_buy_price = price_buy[0]
#         print("---------\n")
#         print("price_buy[0]", price_buy[0])
#         print("price_sell[0]", price_sell[0])
#         grid_sell_price = price_sell[0] if len(price_sell.unique()) == 0 else None #raise ValueError('more than one sell price')
        
#         max_grid_buy_price = max(price_buy)        
#         min_grid_buy_price = min(price_buy)


######### WINNING SOLUTION - START
        self.step -= 1
        if (self.step == 1): return 0
        if (self.step > 1): number_step = min(96, self.step)
        #
        print("\n")
        print(timestamp)
#         print("actual_previous_load", actual_previous_load)
#         print("actual_previous_pv_production", actual_previous_pv_production)

        price_buy = price_buy.tolist()
        price_sell = price_sell.tolist()
        load_forecast = load_forecast.tolist()
        pv_forecast = pv_forecast.tolist() 
        
        # energy: amount of surplus/deficit energy at a specific timestep, depending on actual load / pv values
        energy = [None] * number_step

        for i in range(number_step):
            if (pv_forecast[i] >=50): energy[i] = load_forecast[i] - pv_forecast[i]
            else: energy[i] = load_forecast[i]

        #battery
#         print("battery.capacity", battery.capacity)
#         print("battery.charging_efficiency", battery.charging_efficiency)
#         print("battery.discharging_efficiency", battery.discharging_efficiency)
#         print("1. / battery.discharging_efficiency", 1. / battery.discharging_efficiency)
#         print("(current) capacity * battery.current_charge", battery.capacity * battery.current_charge)
#         print("battery.charging_power_limit", battery.charging_power_limit)
#         print("battery.discharging_power_limit", battery.discharging_power_limit)
#         print("battery.current_charge", battery.current_charge)       
        
        capacity = battery.capacity
        charging_efficiency = battery.charging_efficiency
        discharging_efficiency = 1. / battery.discharging_efficiency
        current = capacity * battery.current_charge 
        limit = battery.charging_power_limit
        dis_limit = battery.discharging_power_limit
        limit /= 4.
        dis_limit /= 4.

        # Ortools
        solver = pywraplp.Solver("B", pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
         
        #Variables: all are continous
        # charge: how much PV to use to charge battery at given time step
        charge = [solver.NumVar(0.0, limit, "c"+str(i)) for i in range(number_step)] 
        # dis_charge: how much energy from battery to use meet load restrictions
        dis_charge = [solver.NumVar( dis_limit, 0.0, "d"+str(i)) for i in range(number_step)]
        # battery_power: amount of power a battery has for each timestep
        battery_power = [solver.NumVar(0.0, capacity, "b"+str(i)) for i in range(number_step+1)]
        # grid: amount of energy to get from grid at each timestep
        grid = [solver.NumVar(0.0, solver.infinity(), "g"+str(i)) for i in range(number_step)] 
        
        #Objective function
        objective = solver.Objective()
        for i in range(number_step):
            objective.SetCoefficient(grid[i], price_buy[i] - price_sell[i]) # why is the coefficient the difference bt buy and sell price?
            objective.SetCoefficient(charge[i], price_sell[i] + price_buy[i] / 1000.) # why is the coefficient the sum of buy and sell price?
            objective.SetCoefficient(dis_charge[i], price_sell[i])             
        objective.SetMinimization()
         
        # 3 Constraints
        c_grid = [None] * number_step
        c_power = [None] * (number_step+1)
         
        # first constraint
        c_power[0] = solver.Constraint(current, current)
        c_power[0].SetCoefficient(battery_power[0], 1)
         
        for i in range(0, number_step):
            # second constraint
            c_grid[i] = solver.Constraint(energy[i], solver.infinity())
            c_grid[i].SetCoefficient(grid[i], 1)
            c_grid[i].SetCoefficient(charge[i], -1)
            c_grid[i].SetCoefficient(dis_charge[i], -1)
            # third constraint
            c_power[i+1] = solver.Constraint( 0, 0)
            c_power[i+1].SetCoefficient(charge[i], charging_efficiency)
            c_power[i+1].SetCoefficient(dis_charge[i], discharging_efficiency)
            c_power[i+1].SetCoefficient(battery_power[i], 1)
            c_power[i+1].SetCoefficient(battery_power[i+1], -1)

        #solve the model
        solver.Solve()
        
#         print(grid[0].solution_value())
#         print("\n\n\n")
        if ((energy[0] < 0) & (dis_charge[0].solution_value() >= 0)):
            n = 0
            first = -limit
            mid = 0

            sum_charge = charge[0].solution_value()
            last = energy[0]
            for n in range(1, number_step):
                if((energy[n] > 0) | (dis_charge[n].solution_value() < 0) | (price_sell[n] != price_sell[n-1])):
                    break
                last = min(last, energy[n])
                sum_charge += charge[n].solution_value()
            if (sum_charge <= 0.):
                 return battery_power[1].solution_value() / capacity
            def tinh(X):
                res = 0
                for i in range(n):
                    res += min(limit, max(-X - energy[i], 0.))
                if (res >= sum_charge): return True
                return False 
            last = 2 - last
            # binary search
            while (last - first > 1):
                mid = (first + last) / 2
                if (tinh(mid)): first = mid
                else: last = mid
            print("(current + min(limit, max(-first - energy[0] , 0)) * charging_efficiency) / capacity")
            print((current + min(limit, max(-first - energy[0] , 0)) * charging_efficiency) / capacity)
            return (current + min(limit, max(-first - energy[0] , 0)) * charging_efficiency) / capacity
        
        if ((energy[0] > 0) & (charge[0].solution_value() <=0)):
            n = 0
            first = dis_limit
            mid = 0
            sum_discharge = dis_charge[0].solution_value()
            last = energy[0]
            for n in range(1, number_step):
                if ((energy[n] < 0) | (charge[n].solution_value() > 0) | (price_sell[n] != price_sell[n-1]) | (price_buy[n] != price_buy[n-1])):
                    break
                last = max(last, energy[n])
                sum_discharge += dis_charge[n].solution_value()
            if (sum_discharge >= 0.): 
                print("battery_power[1].solution_value() / capacity", battery_power[1].solution_value() / capacity)
                return battery_power[1].solution_value() / capacity

            def tinh2(X):
                res = 0
                for i in range(n):
                    res += max(dis_limit, min(X - energy[i], 0))
                if (res <= sum_discharge): return True
                return False                      
            last += 2

            # binary search
            while (last - first > 1):
                mid = (first + last) / 2
                if (tinh2(mid)): first = mid
                else: last = mid
            
            print("(current +  max(dis_limit, min(first - energy[0], 0)) * discharging_efficiency) / capacity")
            print((current +  max(dis_limit, min(first - energy[0], 0)) * discharging_efficiency) / capacity)
            return (current +  max(dis_limit, min(first - energy[0], 0)) * discharging_efficiency) / capacity
######### WINNING SOLUTION - END


### BatteryController - Mine

In [6]:
class BatteryContoller(object):
    """ The BatteryContoller class handles providing a new "target state of charge"
        at each time step.
        This class is instantiated by the simulation script, and it can
        be used to store any state that is needed for the call to
        propose_state_of_charge that happens in the simulation.
        The propose_state_of_charge method returns the state of
        charge between 0.0 and 1.0 to be attained at the end of the coming
        quarter, i.e., at time t+15 minutes.
        The arguments to propose_state_of_charge are as follows:
        :param site_id: The current site (building) id in case the model does different work per site
        :param timestamp: The current timestamp inlcuding time of day and date
        :param battery: The battery (see battery.py for useful properties, including current_charge and capacity)
        :param actual_previous_load: The actual load of the previous quarter.
        :param actual_previous_pv_production: The actual PV production of the previous quarter.
        :param price_buy: The price at which electricity can be bought from the grid for the
          next 96 quarters (i.e., an array of 96 values).
        :param price_sell: The price at which electricity can be sold to the grid for the
          next 96 quarters (i.e., an array of 96 values).
        :param load_forecast: The forecast of the load (consumption) established at time t for the next 96
          quarters (i.e., an array of 96 values).
        :param pv_forecast: The forecast of the PV production established at time t for the next
          96 quarters (i.e., an array of 96 values).
        :returns: proposed state of charge, a float between 0 (empty) and 1 (full).
    """
    
    step = 960
    
    def propose_state_of_charge(self,
                                site_id,
                                timestamp,
                                battery,
                                actual_previous_load,
                                actual_previous_pv_production,
                                price_buy,
                                price_sell,
                                load_forecast,
                                pv_forecast):

        # return the proposed state of charge ...

        self.step -= 1
        if (self.step == 1): return 0
        if (self.step > 1): number_step = min(96, self.step)
            
        print(timestamp)
        print("actual_previous_load", actual_previous_load)
        print("actual_previous_pv_production", actual_previous_pv_production)

        price_buy = price_buy.tolist()
        price_sell = price_sell.tolist()
        load_forecast = load_forecast.tolist()
        pv_forecast = pv_forecast.tolist() 

        #battery
#         print("battery.capacity", battery.capacity)
#         print("battery.charging_efficiency", battery.charging_efficiency)
#         print("battery.discharging_efficiency", battery.discharging_efficiency)
#         print("1. / battery.discharging_efficiency", 1. / battery.discharging_efficiency)
#         print("(current) capacity * battery.current_charge", battery.capacity * battery.current_charge)
#         print("battery.charging_power_limit", battery.charging_power_limit)
#         print("battery.discharging_power_limit", battery.discharging_power_limit)
#         print("battery.current_charge", battery.current_charge)       
        
        battery_capacity = battery.capacity
        charging_efficiency = battery.charging_efficiency
        discharging_efficiency = battery.discharging_efficiency
        current = battery_capacity * battery.current_charge 
        charge_limit = battery.charging_power_limit
        discharge_limit = battery.discharging_power_limit
        charge_limit /= 4.
        discharge_limit /= 4.

        print("before setting solver")
        # Ortools
        solver = pywraplp.Solver("BatteryCharge", pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
         
        # charge: how much PV to use to charge battery at given time step
        print("before setting charge")
        charge = [solver.NumVar(0.0, charge_limit, "charge_{}".format(i)) for i in range(number_step)] 
        # discharge: how much energy from battery to use meet load restrictions
        #discharge = [solver.NumVar( discharge_limit, 0.0, "discharge_{}".format(i)) for i in range(number_step)]
        # battery_power: amount of power a battery has for each timestep
        print("before setting battery_power")
        battery_power = [solver.NumVar(0.0, battery_capacity, "battery_charge_{}".format(i)) for i in range(number_step + 1)]
        
        # grid_energy: amount of energy to get from grid at each timestep
        print("before setting grid_energy")
        grid_energy = [solver.NumVar(0.0, solver.infinity(), "grid_".format(i)) for i in range(number_step)] 
        
        # energy_reqd: amount of surplus/deficit energy at a specific timestep, depending on actual load / pv values
        
        print("before setting energy-reqd")
        energy_reqd = [None] * number_step

        for i in range(number_step):
            energy_reqd[i] = load_forecast[i] - pv_forecast[i]
            
        #energy_to_buy = [solver.NumVar(0.0, solver.infinity(), "energy_to_buy_{}".format(i)) for i in range(number_step)] 
        print("before setting energy_to_sell")
        energy_to_sell = [solver.NumVar(discharge_limit, 0, "energy_to_sell_{}".format(i)) for i in range(number_step)]
        
        #Objective function
        objective = solver.Objective()
        print("before loop to set coefficient on objective")
        for i in range(number_step):
            objective.SetCoefficient(grid_energy[i], price_buy[i] - price_sell[i])
            objective.SetCoefficient(charge[i], price_buy[i])
            objective.SetCoefficient(energy_to_sell[i], -price_sell[i])
            #objective.SetCoefficient(energy_to_buy[i], price_buy[i])
            #objective.SetCoefficient(energy_to_sell[i], -price_sell[i])        
        objective.SetMinimization()
        
        grid_constraint = [None] * number_step
        battery_constraint = [None] * (number_step + 1)
        battery_constraint[0] = solver.Constraint(current, current)
        battery_constraint[0].SetCoefficient(battery_power[0], 1)
        
        print("before loop to set constraints")
        for i in range(0, number_step):
            
            print("before setting coefficient on grid")
            grid_constraint[i] = solver.Constraint(energy_reqd[i], solver.infinity())
            grid_constraint[i].SetCoefficient(grid_energy[i], 1)
            ##grid_constraint[i].SetCoefficient(energy_to_sell[i], -1)
            grid_constraint[i].SetCoefficient(charge[i], -1)
            grid_constraint[i].SetCoefficient(energy_to_sell[i], -1)
            
            print("before setting coefficient on battery")
            battery_constraint[i + 1] = solver.Constraint(0, 0)
            battery_constraint[i + 1].SetCoefficient(charge[i], charging_efficiency)
            battery_constraint[i + 1].SetCoefficient(discharge[i], discharging_efficiency)
            battery_constraint[i + 1].SetCoefficient(battery_power[i], 1)
            battery_constraint[i + 1].SetCoefficient(battery_power[i + 1], -1)
            # second constraint
        
            
            #c_grid[i].SetCoefficient(grid[i], 1)
            #c_grid[i].SetCoefficient(charge[i], -1)
            #c_grid[i].SetCoefficient(dis_charge[i], -1)
            # third constraint

            #c_power[i+1] = solver.Constraint( 0, 0)
            #c_power[i+1].SetCoefficient(charge[i], charging_efficiency)
            #c_power[i+1].SetCoefficient(dis_charge[i], discharging_efficiency)
            #c_power[i+1].SetCoefficient(battery_power[i], 1)
            #c_power[i+1].SetCoefficient(battery_power[i+1], -1)

        #solve the model
        
        print("before solving model")
        solver.Solve()
        
        print(grid_energy[0].solution_value())
        print("\n\n\n")
        return 1.0

In [ ]:
from ortools.linear_solver import pywraplp

# from simulate.battery import Battery
# from simulate.simulate import Simulation

#for site_id, parameters in tqdm(metadata[0:1].iterrows(), desc='sites\t\t\t', total=1): #metadata.shape[0]):
for site_id, parameters in metadata[0:1].iterrows():    
    site_data_path = os.path.join(data_dir, "submit", "{}.csv".format(site_id))

    if os.path.exists(site_data_path):
        site_data = pd.read_csv(site_data_path,
            parse_dates=['timestamp'],
            index_col='timestamp')
        
    #for batt_id in tqdm([1, 2], desc=' > batteries \t\t'):
    for batt_id in [1]:
        print("setting Battery")
        batt = Battery(capacity=parameters[f"Battery_{batt_id}_Capacity"] * 1000,
                       charging_power_limit=parameters[f"Battery_{batt_id}_Power"] * 1000,
                       discharging_power_limit=-parameters[f"Battery_{batt_id}_Power"] * 1000,
                       charging_efficiency=parameters[f"Battery_{batt_id}_Charge_Efficiency"],
                       discharging_efficiency=parameters[f"Battery_{batt_id}_Discharge_Efficiency"])
        
        n_periods = site_data.period_id.nunique()
        #for g_id, g_df in tqdm(site_data.groupby('period_id')[0:1], total=n_periods, desc=' > > periods\t\t'):
        for period in site_data.period_id.unique()[0:1]:
            print("site_id", site_id)
            print("period", period)
            g_id = period
            g_df = site_data[site_data["period_id"] == g_id].sort_index()

        #or g_id, g_df in site_data.groupby('period_id')[0:1]:   
            subset_df = g_df[['period_id', 'site_id', 'actual_consumption', 'actual_pv']]
            
            # reset battery to no charge before simulation
            batt.current_charge = 0
            sim = Simulation(g_df, batt, site_id)
            money_spent, money_no_batt = sim.run()


In [ ]:
site_data.groupby('period_id')

In [ ]:
54760.3223404 - 9291.34509533